In [1]:
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt
pd.options.display.max_rows=100
import metrics
from catboost import CatBoostRegressor, Pool
from  collections import Counter
import numpy as np
import itertools
pd.options.mode.chained_assignment = None
import tensorflow
import math

In [2]:
data_path=Path('data')
users = pd.read_csv(data_path / 'users.tsv', delimiter='\t')
history = pd.read_csv(data_path / 'history.tsv', delimiter='\t')
validate = pd.read_csv(data_path / 'validate.tsv', delimiter='\t')
validate_answers = pd.read_csv(data_path / 'validate_answers.tsv', delimiter='\t')
cpm_probs = pd.read_csv(data_path / 'cpm_probs.tsv', delimiter='\t', index_col='cpm')



In [3]:
import tensorflow as tf

def get_smoothed_mean_log_accuracy_ratio(answers, responses):
    log_accuracy_ratio_mean = tf.math.reduce_mean(
        [
            get_smoothed_log_mape_column_value(responses[:,0], answers[:,0]),
            get_smoothed_log_mape_column_value(responses[:,1], answers[:,1]),
            get_smoothed_log_mape_column_value(responses[:,2], answers[:,2]),
        ]
    )

    percentage_error = 100 * (tf.math.exp(log_accuracy_ratio_mean) - 1)

    return percentage_error

def get_smoothed_log_mape_column_value(y_true, y_pred):
    epsilon=0.005
    return  tf.math.reduce_mean(tf.math.abs(tf.math.log(
        (y_true + epsilon)
        / (y_pred + epsilon)
    )))


In [4]:
hour_split = validate.hour_start.median()+100
# hour_split = validate.hour_start.max()+1

In [5]:
validate_train = validate[validate.hour_start<=hour_split]
validate_val = validate[validate.hour_start>hour_split]
validate_answers_train = validate_answers.loc[validate_train.index]
validate_answers_val = validate_answers.iloc[validate_val.index]
# history_train = history[history.hour<=hour_split]
len(validate_train)

666

In [6]:
def list_as_ints(l):
    return [int(i) for i in l]
    
def get_users_pub_activity(x, d, f=None):
    s = 0
    for k in itertools.product(list_as_ints(x[0].split(',')), list_as_ints(x[1].split(','))):
        s+= d.get(k, 0)
    if f is not None:
        s = f(s)
    return s

def get_users_pub_activity_list(x, d, f=None):
    s = {i:0 for i in list_as_ints(x[1].split(','))}
    for k in itertools.product(list_as_ints(x[0].split(',')), list_as_ints(x[1].split(','))):
        s[k[1]] += d.get(k, 0)
    return list(s.values())


# activities_range = [40, 75, 125, 175, 250, 325, 1000]
# activities_range = [31, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325,  1000]
# activities_range = [31, 75, 125, 175, 250, 325, 1000] 
activities_range = [31, 75, 125, 175, 250, 1000] 
# activities_range = [31, 75, 125, 250, 1000]

quantilies_ranges = range(5, 101, 2)
users_city = users.city_id.to_dict()


    
    
    
for v in [validate_train, validate_val]:
    if len(v) == 0:
        continue
    min_hour=v.hour_start.min()
    history_train = history[history.hour<hour_split]

    users_pub_activities = {}
    for i in activities_range:
        users_pub_activities[i] = history_train[history_train.cpm<i].groupby(['publisher', 'user_id'])['hour'].count().to_dict()
#         users_pub_activities_w[i] = history_train[history_train.cpm<i].groupby(['publisher', 'user_id'])['hour'].min().map(lambda x: 1/x).to_dict()

#     users_count = Counter(np.hstack(validate['user_ids'].map(lambda x: np.array(x.split(','))).values))
    
#     for i in range(5,30,5):
#         v['users_count' + str(i)]=v.user_ids.map(lambda x: np.sum([users_count.get(x, 0) < i for x in x.split(',')]))
#         v['users_count_percent' + str(i)]=v['users_count' + str(i)]/(v['audience_size']+0.0001)


    
    users_in_history = set(history_train[history_train.hour<min_hour].user_id.values)


    v['hours_diff'] = v.hour_end - v.hour_start
    v['hours_diff2']=v['hours_diff']*v['hours_diff']
    validate_cities = validate.user_ids.map(lambda x: Counter([users_city[int(i)] for i in x.split(',')]))

    v['msk_percent'] = validate_cities.map(lambda x: x[0]/sum(x.values()))
    v['spb_percent'] = validate_cities.map(lambda x: x[3]/sum(x.values()))
    v['ekb_percent'] = validate_cities.map(lambda x: x[7]/sum(x.values()))
#     v['hours_day'] = v.hour_end%24
    
    act_lists = v[['publishers', 'user_ids']].apply(lambda x: get_users_pub_activity_list(x, users_pub_activities[1000]), axis=1)
    for i in quantilies_ranges:
        v['activity_q' + str(i)] = [np.quantile(l, i/100) for l in act_lists]
    
#     act_lists_100 = v[['publishers', 'user_ids']].apply(lambda x: get_users_pub_activity_list(x, users_pub_activities[100]), axis=1)
#     for i in quantilies_ranges:
#         v['activity_100_q' + str(i)] = [np.quantile(l, i/100) for l in act_lists_100]
        
    for i in activities_range:
#         v['activity2_' + str(i)] = v[['publishers', 'user_ids']].apply(lambda x : get_users_pub_activity(x,users_pub_activities_w[i]), axis=1)

        v['activity_' + str(i)] = v[['publishers', 'user_ids']].apply(lambda x : get_users_pub_activity(x,users_pub_activities[i]), axis=1)
        v['activity_log_' + str(i)] = v[['publishers', 'user_ids']].apply(lambda x : get_users_pub_activity(x,users_pub_activities[i], math.log), axis=1)
        v['activity*activity_log_' + str(i)]= v['activity_' + str(i)]*v['activity_log_' + str(i)]
        

        v['activity/audience_size_' + str(i)]= v['activity_' + str(i)]/(v['audience_size']+0.0001)
#         v['activity/audience_size**2_' + str(i)]= v['activity_' + str(i)]/(v['audience_size']+0.0001)**2
        v['activity*hours_diff_' + str(i)]= v['activity_' + str(i)]*v['hours_diff']
        v['activity*hours_diff/audience_size_' + str(i)]= v['activity_' + str(i)]*v['hours_diff']/(v['audience_size']+0.0001)
        v['activity*hours_diff*cpm_' + str(i)]= v['activity_' + str(i)]*v['hours_diff']*v['cpm']
        v['activity*hours_diff*cpm/audience_size_' + str(i)]= v['activity_' + str(i)]*v['hours_diff']*v['cpm']/(v['audience_size']+0.0001)

#     for i in activities_range[:-1]:
#         v['activity*activity_' + str(i)]= (v['activity_' + str(i)]*v['activity_1000']).map(lambda x : math.log(x))
     
#      for i in activities_range[:-2]:
#         v['activity*activity_' + str(i)]= (v['activity_' + str(i)]*v['activity_325']).map(lambda x : math.log(x))

#     v['activity_product']=1
#     v['activity_sum']=0
#     for i in activities_range:
#         v['activity_product']= v['activity_product']*v['activity_' + str(i)]
#         v['activity_sum']= v['activity_sum']+v['activity_' + str(i)]

In [7]:
if len(validate_val)  ==0:
    validate_val = validate_train.copy()
    validate_answers_val = validate_answers_train.copy()
   

In [8]:
train_columns = [
    'cpm', 
    'hours_diff', 

      ]


train_columns3=[ ]
for i in activities_range:
#     train_columns3.append('activity2_' + str(i))
    train_columns3.append('activity_' + str(i))
    train_columns3.append('activity_log_' + str(i))
    train_columns3.append('activity/audience_size_' + str(i))
    train_columns3.append('activity*hours_diff_' + str(i))
    train_columns3.append('activity*hours_diff*cpm_' + str(i))
    train_columns3.append('activity*hours_diff*cpm/audience_size_' + str(i))
    
    
train_columns4=['msk_percent', 'spb_percent']




train_columns2= ['activity_q' + str(i) for i in quantilies_ranges]
# +['activity_100_q' + str(i) for i in quantilies_ranges]
# train_columns2.append('users_count')


In [9]:
m = {}
s = {}
for c in train_columns+train_columns2+train_columns3 + train_columns4:
    x = np.concatenate([validate_train[c].values, validate_val[c].values ])
    s[c] = np.max(x)
    
for v in [validate_train, validate_val]:
    for c in set(train_columns +train_columns2+train_columns3+train_columns4):
        v[c]=(v[c])/s[c]



In [10]:

validate_train[train_columns].max().max(),\
                 validate_train[train_columns2].max().max(),\
                 validate_train[train_columns3].max().max(),\
                 validate_train[train_columns4].max().max(),


(1.0, 1.0, 1.0, 1.0)

In [11]:
# train_columns

In [ ]:
def model_block(l1, l2, l3, l4):
    kernel_initializer='lecun_uniform'
#     kernel_initializer=None
    from tensorflow.keras import regularizers
    kernel_regularizer=regularizers.l2(0.001)
    kernel_regularizer=None

    activity_regularizer=regularizers.l1(0.01)



    x4 = Dense(64, activation='relu', kernel_initializer=kernel_initializer)(l4)
    x4 = Dropout(0.5)(x4)
    x4 = Dense(8)(x4)
#     l3 = Dropout(0.05)(l3)
    x3 = Dense(256, activation='relu', kernel_initializer=kernel_initializer)(l3)
    x3 = Dropout(0.4)(x3)
    x3 = Dense(48)(x3)
    
    l2 = Dropout(0.3)(l2)
    x2 = Dense(128, activation='relu', kernel_initializer=kernel_initializer)(l2)
    x2 = Dense(16)(x2) 
        
    x = Dense(128, activation='relu', kernel_initializer=kernel_initializer)(l1)
    x = Dropout(0.2)(x)
    x = Dense(32)(x)
        
    concat = Concatenate()([x2, x3,x])
    concat = Dense(48)(concat)
    concat = Dense(32, activation='relu')(concat)
#     concat = Dense(8, activation='sigmoid')(concat)
    return Dense(3, activation='sigmoid', kernel_regularizer=kernel_regularizer)(concat)
def get_model():
    in_ = Input(shape=(len(train_columns),))
    in2_ = Input(shape=(len(train_columns2)))
    in3_ = Input(shape=(len(train_columns3)))
    in4_ = Input(shape=(len(train_columns4)))




    outputs = [model_block(in_, in2_, in3_, in4_) for _ in range(10)]

    model = Model([in_, in2_, in3_, in4_], outputs)
    return model

for i in range(1):
    from tensorflow.keras.layers import Input, Dense,Concatenate, Dropout,multiply
    from tensorflow.keras import Model
    from tensorflow.keras import optimizers
    fields = ['at_least_one', 'at_least_two', 'at_least_three']

    model = get_model()

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        0.001, #->  down
        decay_steps=200,
        decay_rate=0.95)
             
    adam = optimizers.Adam(learning_rate=lr_schedule, beta_1=0.85, beta_2=0.85)
#     adam = optimizers.RMSprop(learning_rate=lr_schedule, rho=0.5)
    model.compile(optimizer=adam, loss=get_smoothed_mean_log_accuracy_ratio)
    model.fit(x=[validate_train[train_columns].values,
                 validate_train[train_columns2].values,
                 validate_train[train_columns3].values,
                 validate_train[train_columns4].values,
                 

                ],
              
                y=[validate_answers_train[fields].values]*len(model.outputs),
                validation_data=([validate_val[train_columns].values,
                                  validate_val[train_columns2].values,
                                  validate_val[train_columns3].values,
                                  validate_val[train_columns4].values,
                                 

                                 ], [validate_answers_val[fields]]*len(model.outputs)),
                epochs=4000,
                batch_size = len(validate_train)
            )
    

    validate_val.loc[:,'at_least_one']=np.median(model.predict(  [validate_val[train_columns].values,validate_val[train_columns2].values,validate_val[train_columns3].values,validate_val[train_columns4].values]), axis=0)[:,0]
    validate_val.loc[:,'at_least_two']=np.median(model.predict(  [validate_val[train_columns].values,validate_val[train_columns2].values,validate_val[train_columns3].values,validate_val[train_columns4].values]), axis=0)[:,1]
    validate_val.loc[:,'at_least_three']=np.median(model.predict([validate_val[train_columns].values,validate_val[train_columns2].values,validate_val[train_columns3].values,validate_val[train_columns4].values]), axis=0)[:,2]
    print(metrics.get_smoothed_mean_log_accuracy_ratio(validate_answers_val, validate_val))
    validate_val.loc[:,'at_least_one']=np.mean(model.predict([validate_val[train_columns].values,validate_val[train_columns2].values,validate_val[train_columns3].values,validate_val[train_columns4].values]), axis=0)[:,0]
    validate_val.loc[:,'at_least_two']=np.mean(model.predict([validate_val[train_columns].values,validate_val[train_columns2].values,validate_val[train_columns3].values,validate_val[train_columns4].values]), axis=0)[:,1]
    validate_val.loc[:,'at_least_three']=np.mean(model.predict([validate_val[train_columns].values,validate_val[train_columns2].values,validate_val[train_columns3].values,validate_val[train_columns4].values]), axis=0)[:,2]
    print(metrics.get_smoothed_mean_log_accuracy_ratio(validate_answers_val, validate_val))


Train on 666 samples, validate on 342 samples
Epoch 1/4000
666/666 [==============================] - 5s 8ms/sample - loss: 13807.3008 - dense_10_loss: 1465.8035 - dense_21_loss: 1186.3906 - dense_32_loss: 1333.6484 - dense_43_loss: 1262.2700 - dense_54_loss: 1454.8030 - dense_65_loss: 1392.3712 - dense_76_loss: 1374.8599 - dense_87_loss: 1387.9871 - dense_98_loss: 1295.1393 - dense_109_loss: 1654.0277 - val_loss: 24235.1484 - val_dense_10_loss: 2630.3149 - val_dense_21_loss: 2017.6067 - val_dense_32_loss: 2466.4983 - val_dense_43_loss: 2271.7649 - val_dense_54_loss: 2561.5684 - val_dense_65_loss: 2449.5010 - val_dense_76_loss: 2447.0696 - val_dense_87_loss: 2502.8455 - val_dense_98_loss: 2068.2126 - val_dense_109_loss: 2819.7671
Epoch 2/4000
666/666 [==============================] - 0s 52us/sample - loss: 12324.9375 - dense_10_loss: 1329.2821 - dense_21_loss: 1038.5803 - dense_32_loss: 1223.1550 - dense_43_loss: 1127.2400 - dense_54_loss: 1285.8671 - dense_65_loss: 1252.3230 - dense_

Epoch 13/4000
666/666 [==============================] - 0s 48us/sample - loss: 4725.1279 - dense_10_loss: 498.9875 - dense_21_loss: 380.2713 - dense_32_loss: 474.4716 - dense_43_loss: 385.8660 - dense_54_loss: 400.8589 - dense_65_loss: 414.4604 - dense_76_loss: 492.2240 - dense_87_loss: 518.0713 - dense_98_loss: 430.7529 - dense_109_loss: 729.1642 - val_loss: 4816.2100 - val_dense_10_loss: 546.2849 - val_dense_21_loss: 294.7968 - val_dense_32_loss: 479.9441 - val_dense_43_loss: 307.6794 - val_dense_54_loss: 373.9540 - val_dense_65_loss: 387.7679 - val_dense_76_loss: 518.6625 - val_dense_87_loss: 580.4986 - val_dense_98_loss: 297.7799 - val_dense_109_loss: 1028.8424
Epoch 14/4000
666/666 [==============================] - 0s 47us/sample - loss: 4489.5278 - dense_10_loss: 461.0670 - dense_21_loss: 368.0123 - dense_32_loss: 455.4866 - dense_43_loss: 368.1799 - dense_54_loss: 391.5427 - dense_65_loss: 391.1861 - dense_76_loss: 460.2266 - dense_87_loss: 484.7633 - dense_98_loss: 421.6073 -

Epoch 26/4000
666/666 [==============================] - 0s 44us/sample - loss: 2680.8569 - dense_10_loss: 260.6061 - dense_21_loss: 253.7209 - dense_32_loss: 269.1492 - dense_43_loss: 235.5453 - dense_54_loss: 283.8130 - dense_65_loss: 299.1321 - dense_76_loss: 275.3829 - dense_87_loss: 260.5993 - dense_98_loss: 235.3651 - dense_109_loss: 307.5431 - val_loss: 2154.4294 - val_dense_10_loss: 221.4159 - val_dense_21_loss: 193.1414 - val_dense_32_loss: 199.0112 - val_dense_43_loss: 180.4836 - val_dense_54_loss: 169.9913 - val_dense_65_loss: 199.4550 - val_dense_76_loss: 201.3357 - val_dense_87_loss: 209.4437 - val_dense_98_loss: 255.1768 - val_dense_109_loss: 324.9748
Epoch 27/4000
666/666 [==============================] - 0s 43us/sample - loss: 2561.3740 - dense_10_loss: 243.7097 - dense_21_loss: 240.4140 - dense_32_loss: 250.9257 - dense_43_loss: 225.4283 - dense_54_loss: 281.6437 - dense_65_loss: 286.0120 - dense_76_loss: 260.6616 - dense_87_loss: 253.9147 - dense_98_loss: 226.4526 - 

Epoch 39/4000
666/666 [==============================] - 0s 43us/sample - loss: 1570.6514 - dense_10_loss: 140.3179 - dense_21_loss: 134.2579 - dense_32_loss: 172.5562 - dense_43_loss: 159.4111 - dense_54_loss: 178.2053 - dense_65_loss: 173.6065 - dense_76_loss: 139.5757 - dense_87_loss: 148.9052 - dense_98_loss: 132.4156 - dense_109_loss: 191.4000 - val_loss: 1306.6350 - val_dense_10_loss: 128.9375 - val_dense_21_loss: 128.6589 - val_dense_32_loss: 117.8030 - val_dense_43_loss: 113.3990 - val_dense_54_loss: 137.8753 - val_dense_65_loss: 155.3952 - val_dense_76_loss: 127.7729 - val_dense_87_loss: 133.4408 - val_dense_98_loss: 118.3248 - val_dense_109_loss: 145.0276
Epoch 40/4000
666/666 [==============================] - 0s 51us/sample - loss: 1518.6654 - dense_10_loss: 137.0008 - dense_21_loss: 130.7956 - dense_32_loss: 175.2953 - dense_43_loss: 155.8397 - dense_54_loss: 167.3817 - dense_65_loss: 163.2898 - dense_76_loss: 138.6090 - dense_87_loss: 140.5373 - dense_98_loss: 127.3832 - 

Epoch 52/4000
666/666 [==============================] - 0s 42us/sample - loss: 1067.1052 - dense_10_loss: 103.1229 - dense_21_loss: 96.1292 - dense_32_loss: 119.4326 - dense_43_loss: 98.8273 - dense_54_loss: 112.9340 - dense_65_loss: 104.0912 - dense_76_loss: 104.0501 - dense_87_loss: 103.2699 - dense_98_loss: 101.6038 - dense_109_loss: 123.6442 - val_loss: 910.1136 - val_dense_10_loss: 89.6018 - val_dense_21_loss: 85.7775 - val_dense_32_loss: 91.3466 - val_dense_43_loss: 82.9058 - val_dense_54_loss: 98.2294 - val_dense_65_loss: 92.5816 - val_dense_76_loss: 88.0876 - val_dense_87_loss: 88.2405 - val_dense_98_loss: 84.2307 - val_dense_109_loss: 109.1121
Epoch 53/4000
666/666 [==============================] - 0s 44us/sample - loss: 1036.6674 - dense_10_loss: 100.7944 - dense_21_loss: 97.8487 - dense_32_loss: 113.9853 - dense_43_loss: 94.1535 - dense_54_loss: 110.8692 - dense_65_loss: 97.7979 - dense_76_loss: 101.5841 - dense_87_loss: 99.5900 - dense_98_loss: 97.9495 - dense_109_loss: 1

...

Epoch 3198/4000
666/666 [==============================] - 0s 43us/sample - loss: 99.5145 - dense_10_loss: 9.7488 - dense_21_loss: 10.0074 - dense_32_loss: 10.0387 - dense_43_loss: 9.6438 - dense_54_loss: 9.8364 - dense_65_loss: 10.1617 - dense_76_loss: 10.0974 - dense_87_loss: 10.2744 - dense_98_loss: 9.8590 - dense_109_loss: 9.8469 - val_loss: 132.0059 - val_dense_10_loss: 12.8109 - val_dense_21_loss: 13.3306 - val_dense_32_loss: 13.4613 - val_dense_43_loss: 13.4103 - val_dense_54_loss: 13.4228 - val_dense_65_loss: 13.2475 - val_dense_76_loss: 13.1949 - val_dense_87_loss: 13.4206 - val_dense_98_loss: 13.0305 - val_dense_109_loss: 12.6765
Epoch 3199/4000
666/666 [==============================] - 0s 51us/sample - loss: 99.4931 - dense_10_loss: 9.6708 - dense_21_loss: 10.0602 - dense_32_loss: 10.2175 - dense_43_loss: 10.0194 - dense_54_loss: 9.7057 - dense_65_loss: 10.3752 - dense_76_loss: 9.6524 - dense_87_loss: 10.0506 - dense_98_loss: 9.8213 - dense_109_loss: 9.9199 - val_loss: 132.

Epoch 3211/4000
666/666 [==============================] - 0s 50us/sample - loss: 99.9981 - dense_10_loss: 9.7544 - dense_21_loss: 10.2427 - dense_32_loss: 9.9136 - dense_43_loss: 9.9255 - dense_54_loss: 9.7983 - dense_65_loss: 10.0873 - dense_76_loss: 9.9129 - dense_87_loss: 10.5940 - dense_98_loss: 9.9697 - dense_109_loss: 9.7997 - val_loss: 132.6558 - val_dense_10_loss: 13.2588 - val_dense_21_loss: 13.2325 - val_dense_32_loss: 13.4548 - val_dense_43_loss: 13.4667 - val_dense_54_loss: 13.2859 - val_dense_65_loss: 13.0252 - val_dense_76_loss: 13.1524 - val_dense_87_loss: 13.5286 - val_dense_98_loss: 13.1759 - val_dense_109_loss: 13.0750
Epoch 3212/4000
666/666 [==============================] - 0s 50us/sample - loss: 99.4447 - dense_10_loss: 9.9339 - dense_21_loss: 9.7979 - dense_32_loss: 10.2082 - dense_43_loss: 10.0550 - dense_54_loss: 10.0564 - dense_65_loss: 10.1495 - dense_76_loss: 9.7902 - dense_87_loss: 9.9112 - dense_98_loss: 9.5779 - dense_109_loss: 9.9646 - val_loss: 132.531

Epoch 3224/4000
666/666 [==============================] - 0s 45us/sample - loss: 99.9376 - dense_10_loss: 9.9047 - dense_21_loss: 9.9824 - dense_32_loss: 10.5111 - dense_43_loss: 10.0763 - dense_54_loss: 9.8994 - dense_65_loss: 10.1523 - dense_76_loss: 9.8096 - dense_87_loss: 10.2459 - dense_98_loss: 9.5786 - dense_109_loss: 9.7773 - val_loss: 132.7517 - val_dense_10_loss: 12.9438 - val_dense_21_loss: 13.2391 - val_dense_32_loss: 13.5641 - val_dense_43_loss: 13.4696 - val_dense_54_loss: 13.5934 - val_dense_65_loss: 13.1545 - val_dense_76_loss: 13.2887 - val_dense_87_loss: 13.4595 - val_dense_98_loss: 12.9809 - val_dense_109_loss: 13.0580
Epoch 3225/4000
666/666 [==============================] - 0s 56us/sample - loss: 101.0909 - dense_10_loss: 9.9038 - dense_21_loss: 10.4965 - dense_32_loss: 10.3147 - dense_43_loss: 9.9727 - dense_54_loss: 9.9629 - dense_65_loss: 10.1503 - dense_76_loss: 9.9788 - dense_87_loss: 10.3238 - dense_98_loss: 9.6523 - dense_109_loss: 10.3351 - val_loss: 132.

Epoch 3237/4000
666/666 [==============================] - 0s 44us/sample - loss: 99.7706 - dense_10_loss: 10.1339 - dense_21_loss: 10.1142 - dense_32_loss: 10.1389 - dense_43_loss: 10.2758 - dense_54_loss: 9.5587 - dense_65_loss: 10.0569 - dense_76_loss: 9.9183 - dense_87_loss: 9.9812 - dense_98_loss: 9.6891 - dense_109_loss: 9.9036 - val_loss: 131.6995 - val_dense_10_loss: 12.8208 - val_dense_21_loss: 13.1283 - val_dense_32_loss: 13.5368 - val_dense_43_loss: 13.3341 - val_dense_54_loss: 13.3028 - val_dense_65_loss: 13.1522 - val_dense_76_loss: 13.4017 - val_dense_87_loss: 13.2927 - val_dense_98_loss: 13.0177 - val_dense_109_loss: 12.7126
Epoch 3238/4000
666/666 [==============================] - 0s 44us/sample - loss: 98.8639 - dense_10_loss: 9.7716 - dense_21_loss: 9.9014 - dense_32_loss: 9.9843 - dense_43_loss: 9.9159 - dense_54_loss: 9.4409 - dense_65_loss: 10.4102 - dense_76_loss: 9.6654 - dense_87_loss: 9.7089 - dense_98_loss: 10.0135 - dense_109_loss: 10.0518 - val_loss: 131.25

Epoch 3250/4000
666/666 [==============================] - 0s 51us/sample - loss: 98.6473 - dense_10_loss: 9.9643 - dense_21_loss: 10.1726 - dense_32_loss: 10.0868 - dense_43_loss: 9.7563 - dense_54_loss: 9.6879 - dense_65_loss: 9.8627 - dense_76_loss: 9.7527 - dense_87_loss: 9.8518 - dense_98_loss: 9.8099 - dense_109_loss: 9.7023 - val_loss: 131.9469 - val_dense_10_loss: 13.0125 - val_dense_21_loss: 13.1004 - val_dense_32_loss: 13.5879 - val_dense_43_loss: 13.3855 - val_dense_54_loss: 13.2358 - val_dense_65_loss: 13.1813 - val_dense_76_loss: 13.0629 - val_dense_87_loss: 13.4192 - val_dense_98_loss: 13.0868 - val_dense_109_loss: 12.8745
Epoch 3251/4000
